In [8]:
# Create a nested function 
# See further details here: https://stackabuse.com/python-nested-functions/

# Start small and go slow. Test every step (can use print command to test if working) - Begin by testing the outermost function then generate an inner function ****

def f_test():
    print('f_test is working') # When the function, f_test, is called, print out the following (when printing words rather than defined variables, you must surroung with '')
# f_test(0)
    def f_test_of_inner(): # A nested/inner function 
        print('inner func is working')
    f_test_of_inner() # The inner function must be called after definition. If not, it will not return what you desire (here, it will not print)
f_test() # Call the primary/outer function to recieve the outer and inner return values 

f_test is working
inner func is working


In [9]:
# Using nested functions that rely on one another for completion  


# The nested func. contains your primary interest (i.e., equation computation)
def f_test(x): 
    def f_relying_on_primary_func(y):
        return (y / x)
    return(f_relying_on_primary_func)
answer = f_test(2) # Fixing x as a value of 5 when f_test is called by the variable, answer
print(answer(4)) # When answer is called, it has a pre-defined value for the primary/outer function. Therefore, the number within () defines for the inner/nested function's variable, y


# The primary function relies on a value localted in the nested function
def f_test_2():
    x = 2 # Use single = here. If not, get errors regarding 'cannot access..'
    def f_relying_on_primary_func_2():
        y = 2
        return(y)
    y = f_relying_on_primary_func_2() # Necessary to define the return value as the same one to be used in primary function 
    return(x * y)

print(f_test_2())

2.0
4


In [ ]:
# Creating nested functions where the primary/outer func. depends on the inner/nested, which returns values according to a loop output 

def f_test():
    x = 2 
    def f_looping_an_inner_functions_return():
        if x == 2: # IF statements require a hard assignment using ==
            return(8)
        elif x == 4:
            return(16)
        else: # Indication that the function is not working as it should per the conditions set
            return('nested function did not recieve a numerical input')
    y = f_looping_an_inner_functions_return()
    return(y)

print(f_test())


In [ ]:
# An example of how to use all the above information with a personal research specific example regarding rate constant determination using an ODE model. 

# AIM: Create nested functions where the primary func. relies on the output of the inner func., which is dependent on a loop's output 

# Introducing new points ranges based on a new, complicated model involving movement 1. out of PSII, 2. PQ mediated transfer
# 3. Time spent in Cyt 4. PC mediated transfer 5. out of PSI
        # Ranges defined by subtraces shown above


def f_concentration_change(y, t, PAR, PQred, PQox, PQH2red, PQH2ox, PCred, PCox, Fdred, Fdox, k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, k11):

    A = (k1*y[1] - k1*y[0]*PAR) # PAR activation of the system 
        # Reliant on second order reaction whereby movement from A -> A* is dependent on both PAR and a rate constant 
        # And, there is possibility for the back reaction with dependency on the back rxn rate constant (here, k1, in equil. with the forward) due to an assumption that if A* exists at too high a conc. compared to A, the energy is likely to dissipate as NPQ rather than go back.
# 'Out PSII'
    A_star = -A + (k3*y[2]*PQox) - (k2*y[1]*PQred)  # Movement of A -> A* (inverse of A). The movement of A -> B, here, the movement of PQ out of PSII binding pocket 
        # Movement  out of PSII is dependent on both quantity of PQ carrier molecules available + it's back reaction (not in equil)
        # k2 = PQ out & k3 = PQ going back 
# 'PQ Transfer'
    B = (k2*y[1]*PQred) - (k3*y[2]*PQox) - (k4*y[2]*PQH2red) + (k5*y[3]*PQH2ox) # Diffusion of PQ through the membrane - rate based on current occupation of cytb6f 
        # Both the change of PQ -> PQH2 (the protonation step) at cytb6f, its reverse + The reverse of A* -> B 
            # Inverse of A*->B
            # The forward rxn that is dependent on its rate constant & current occupation of PQH2 in cytb6f 
                # --> Rate of diffusion NOT based on membrane composition - but on current occupation (# of PQH2 already present in cytb6f)
# 'Cytb6f waiting'
    B_star = (k4*y[2]*PQH2red) - (k5*y[3]*PQH2ox) - (k6*y[3]) + (k7*y[4]) # Time spent in cytb6f
        # The flat line of full reduction whereby PQH2 'sits' in complex 
# 'PC Transfer'
    C = (k6*y[3]) - (k7*y[4]) - (k8*y[4]*PCred) + (k9*y[5]*PCox) # PC diffusion through the membrane - rate based on # of available PC carriers 
        # --> Rate of diffusion NOT based on membrane composition - but on availbility of PC 
# 'PSI Waiting and out'
    D = (k8*y[4]*PCred) - (k9*y[5]*PCox) - (k10*y[5]*Fdred) + (k11*y[6]*Fdox) # Time spent in PSI after arrival - dependent on # of Fd reducants
# 'PSI Waiting and out Reverse'
    D_minus = (k10*y[5]*Fdred) - (k11*y[6]*Fdox) # The reverse rxn occuring of D 
    return ( [A, A_star, B, B_star, C, D, D_minus] )

guess_parameters = [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.1, -0.1]


# Assigning a fit curve to a specified length of raw trace. Cutting out outlying values and/or areas of uninterest
genotypes = ['Col-0']
use_bounds = True
guess_parameters_names = [ 'k1', 'k2', 'k3', 'k4', 'k5', 'k6', 'k7', 'k8', 'k9', 'k10', 'k11', 'A', 'offset']
# My new points ranges based on revised model - based on subtraces in cell above
points_ranges = { 'out_PSII':[310,320], 'PQ_transfer':[321,600], 'Cytb6f':[601,1480], 'PC_transfer':[1481,1525], 'out_PSI':[1526,-1] }

# Formatting for df output
for j, sub_trace_index in enumerate(range(0,number_subtraces)):
    for guess_parameters_name in guess_parameters_names:
        chow_df[guess_parameters_name + '_' + str(sub_trace_index)] = 0
for sub_trace_index in range(number_subtraces):
    chow_df['fit out_PSII ' + str(sub_trace_index)] = 0 # Make this new column empty
    chow_df['fit out_PSII ' + str(sub_trace_index)] = chow_df['fit out_PSII ' + str(sub_trace_index)].astype(object) # Make this new column an object capable of storing series of values 
    chow_df['fit params ' + str(sub_trace_index)] = 0 
    chow_df['fit params ' + str(sub_trace_index)] = chow_df['fit params ' + str(sub_trace_index)].astype(object) # Make this new column an object capable of storing series of values         
chow_df['x_data'] = 0 
chow_df['x_data'] = chow_df['x_data'].astype(object)



def f_ode(t, k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, k11, A, offset): 
    PQred = 0 
    PQox = 0
    PQH2red = 0  
    PQH2ox = 0
    PCred = 0 
    PCox = 0
    Fdred = 0
    Fdox = 0
    def associating_subtrace_with_PAR(j): # Where j is defined as the sub_trace_index in the analysis code I am writing these functions for
        # for sub_trace_index in enumerate(range(0, number_subtraces)):
        if j == 0: 
            return(100) # Representing the corresponding PAR values in MultiSpec protocol used in the subtraces. Each subtrace (i.e., pulse of light for probing) had an increasing light intensity
        elif j == 1: 
            return(200)
        elif j == 2: 
            return(400)
        elif j == 3: 
            return(800)
        elif j == 4:
            return(1600)
        elif j == 5:
            return(3200)
        else:
            print('incorrect')
    PAR = associating_subtrace_with_PAR(j)
    constants = (k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, k11, PAR, PQred, PQox, PQH2red, PQH2ox, PCred, PCox, Fdred, Fdox)
    y0 = [0.0, A, 0.0, 0.0, 0.0, 0.0, 0.0] # Changing amplitude to account for the beginning step I am interested in (A* here)
    y_data = odeint(f_concentration_change, y0, t, args=constants)
    return(y_data[:,1]) + offset # Looking at A_star -->B
